# Fakeout & Reversal - Strategy - Backtesting 

### Import Library

In [104]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [105]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [106]:
data = dataset.copy()

In [107]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [108]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-07 14:15:00,1343.3,1344.4,1342.4,1344.2,5075
2025-02-07 14:20:00,1344.2,1344.7,1343.6,1344.0,4865
2025-02-07 14:25:00,1344.1,1344.2,1342.8,1343.3,5013


In [109]:
def prepare_data(df, lookback=20):
    df["resistance"] = df["High"].rolling(lookback).max().shift(1)
    df["support"] = df["Low"].rolling(lookback).min().shift(1)
    return df

In [110]:
class MainStrategy(Strategy):
    risk_reward_ratio = 3
    risk_percent = 0.02
    max_sl = 3.5
    def init(self):
        super().init()

    def next(self):
        super().next()
        if len(self.data) < 11:
            return
        _time = self.data.index[-1]
        if _time.hour == 14 and _time.minute == 25:
            if self.position:
                self.position.close()
            return
        close = self.data.Close[-1]
        low = self.data.Low[-1]
        high = self.data.High[-1]
        support = self.data.support[-10]
        resistance = self.data.resistance[-10]
        if low < support and close > support:
            if not self.position:
                sl = close - self.max_sl
                tp = close + self.max_sl * self.risk_reward_ratio
                self.buy(size=1, sl=sl, tp=tp)

        elif high > resistance and close < resistance:
            if not self.position:
                # print(f'{_time} - {high} - {resistance} - {close}')
                sl = close + self.max_sl
                tp = close - self.max_sl * self.risk_reward_ratio
                self.sell(size=1, sl=sl, tp=tp)

In [111]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)
# prepared_data = prepared_data.tail(300)

In [112]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()
# bt.plot()
print(stats)

Start                     2020-11-02 10:40:00
End                       2025-02-07 14:45:00
Duration                   1558 days 04:05:00
Exposure Time [%]                   39.648016
Equity Final [$]                   9819.41698
Equity Peak [$]                   10043.60541
Return [%]                           -1.80583
Buy & Hold Return [%]               49.355126
Return (Ann.) [%]                    -0.42202
Volatility (Ann.) [%]                1.130673
Sharpe Ratio                        -0.373247
Sortino Ratio                        -0.55872
Calmar Ratio                        -0.126045
Max. Drawdown [%]                   -3.348177
Avg. Drawdown [%]                   -0.352626
Max. Drawdown Duration     1337 days 01:10:00
Avg. Drawdown Duration       91 days 03:03:00
# Trades                                 1752
Win Rate [%]                        34.417808
Best Trade [%]                       1.160003
Worst Trade [%]                     -0.675459
Avg. Trade [%]                    

In [113]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,-1,25,27,897.73060,901.4,-3.66940,-0.004087,2020-11-02 14:10:00,2020-11-02 14:20:00,None,0 days 00:10:00
1,-1,50,80,908.72730,909.0,-0.27270,-0.000300,2020-11-03 10:35:00,2020-11-03 14:30:00,None,0 days 03:55:00
2,-1,97,103,909.92694,913.5,-3.57306,-0.003927,2020-11-04 10:15:00,2020-11-04 10:45:00,None,0 days 00:30:00
3,1,130,131,912.77375,913.5,0.72625,0.000796,2020-11-04 14:25:00,2020-11-04 14:30:00,None,0 days 00:05:00
4,-1,134,150,913.52586,917.1,-3.57414,-0.003912,2020-11-05 09:05:00,2020-11-05 10:25:00,None,0 days 01:20:00
...,...,...,...,...,...,...,...,...,...,...,...
1747,-1,54035,54036,1324.70247,1328.6,-3.89753,-0.002942,2025-02-04 13:05:00,2025-02-04 13:10:00,None,0 days 00:05:00
1748,-1,54047,54052,1328.70127,1328.5,0.20127,0.000151,2025-02-04 14:05:00,2025-02-04 14:30:00,None,0 days 00:25:00
1749,-1,54068,54103,1334.39956,1335.1,-0.70044,-0.000525,2025-02-05 10:10:00,2025-02-05 14:30:00,None,0 days 04:20:00
1750,-1,54120,54154,1341.69737,1339.4,2.29737,0.001712,2025-02-06 10:15:00,2025-02-06 14:30:00,None,0 days 04:15:00


In [114]:
stats['_trades'].tail(20)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
1732,1,53541,53542,1289.58676,1290.2,0.61324,0.000476,2025-01-14 14:25:00,2025-01-14 14:30:00,None,0 days 00:05:00
1733,-1,53553,53567,1294.61150,1298.3,-3.68850,-0.002849,2025-01-15 09:45:00,2025-01-15 10:55:00,None,0 days 01:10:00
1734,-1,53569,53572,1297.01078,1301.0,-3.98922,-0.003076,2025-01-15 11:05:00,2025-01-15 11:20:00,None,0 days 00:15:00
1735,-1,53579,53593,1296.91081,1297.4,-0.48919,-0.000377,2025-01-15 13:20:00,2025-01-15 14:30:00,None,0 days 01:10:00
1736,-1,53608,53642,1306.00808,1295.7,10.30808,0.007893,2025-01-16 10:05:00,2025-01-16 14:20:00,None,0 days 04:15:00
1737,-1,53657,53680,1309.80694,1313.5,-3.69306,-0.002820,2025-01-17 09:55:00,2025-01-17 13:15:00,None,0 days 03:20:00
1738,-1,53709,53717,1321.30349,1325.2,-3.89651,-0.002949,2025-01-20 10:00:00,2025-01-20 10:40:00,None,0 days 00:40:00
1739,-1,53724,53746,1324.00268,1324.5,-0.49732,-0.000376,2025-01-20 11:15:00,2025-01-20 14:30:00,None,0 days 03:15:00
1740,-1,53759,53787,1329.10115,1319.0,10.10115,0.007600,2025-01-21 09:55:00,2025-01-21 13:40:00,None,0 days 03:45:00
1741,1,53796,53797,1320.99618,1321.3,0.30382,0.000230,2025-01-21 14:25:00,2025-01-21 14:30:00,None,0 days 00:05:00


In [115]:
data.tail(50)

,Open,High,Low,Close,Volume,resistance,support
Date,,,,,,,
2025-02-07 09:05:00,1336.6,1337.5,1336.1,1337.3,3224,1343.7,1335.8
2025-02-07 09:10:00,1337.4,1337.4,1336.4,1336.9,2136,1343.7,1335.8
2025-02-07 09:15:00,1336.9,1338.0,1335.0,1335.2,5915,1343.7,1335.8
2025-02-07 09:20:00,1335.2,1336.6,1335.2,1336.4,3481,1343.7,1335.0
2025-02-07 09:25:00,1336.4,1337.5,1336.2,1337.3,3008,1343.7,1335.0
2025-02-07 09:30:00,1337.3,1339.0,1336.9,1337.8,5315,1343.7,1335.0
2025-02-07 09:35:00,1338.0,1338.5,1337.7,1338.5,1753,1343.7,1335.0
2025-02-07 09:40:00,1338.3,1338.5,1337.0,1337.6,2263,1343.7,1335.0
2025-02-07 09:45:00,1337.7,1337.9,1336.8,1336.9,1861,1343.7,1335.0
